<a href="https://colab.research.google.com/github/sinjulee/chapter1-basic/blob/main/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필수 패키지 설치

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 968 kB 26.6 MB/s 
     |████████████████████████████████| 359 kB 52.1 MB/s 
     |████████████████████████████████| 138 kB 43.7 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 3.6 MB 72.8 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin

# 패키지 불러오기

In [2]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

# zdnet의 kt검색어로 검색한 기사의 url 수집

In [4]:
href = []
title = []
date = []
writer = []
for i in tqdm(range(1, 6)):
    driver.get("https://search.zdnet.co.kr/?kwd=kt&area=&pageno=" + str(i) + "&term=")
    driver.page_source
    soup = BeautifulSoup(driver.page_source, "html.parser")
    for row_article in soup.findAll(class_="left_cont")[0].findAll(class_="assetText"):
        href.append(row_article.findAll("a")[0]["href"])
        title.append(row_article.findAll("h3")[0].getText())
        date.append(
            str(row_article.findAll(class_="byline")[0])
            .split("</span>")[0]
            .split("<span>")[1]
        )
        writer.append(
            str(row_article.findAll(class_="byline")[0])
            .split("</span>")[1]
            .split("</p>")[0]
            .strip()
        )

100%|██████████| 5/5 [00:29<00:00,  5.84s/it]


In [5]:
df = pd.DataFrame()
df["href"] = href
df["title"] = title
df["date"] = date
df["writer"] = writer
df

,href,title,date,writer
0,https://zdnet.co.kr/view/?no=20220308100059,"KT스카이라이프, 산불피해 특별재난지역 방송통신 요금 감면",2022.03.08 10:00,박수형
1,https://zdnet.co.kr/view/?no=20220308093008,"국내 첫 유튜브 드라마, KT시즌에서 전편 무료",2022.03.08 09:30,박수형
2,https://zdnet.co.kr/view/?no=20220308091148,"HCN, 제2 개국 선언...""지역 1등 네트워크 플랫폼""",2022.03.08 09:11,박수형
3,https://zdnet.co.kr/view/?no=20220308090050,"KT, AI 코딩블록 교육 도구에 IoT 더했다",2022.03.08 09:00,박수형
4,https://zdnet.co.kr/view/?no=20220307162912,"""이통 3사 긴급구조 위치정보 품질 개선됐다""",2022.03.07 16:29,박수형
...,...,...,...,...
95,https://zdnet.co.kr/view/?no=20220204163040,통신 3사 작년 영업이익 총합 4조원 안팎될 듯,2022.02.04 16:45,박수형
96,https://zdnet.co.kr/view/?no=20220204094849,"KT, 이투스 메타버스 교육플랫폼에 클라우드 적용",2022.02.04 09:48,박수형
97,https://zdnet.co.kr/view/?no=20220203171909,"임혜숙 장관, 17일 이통 3사 CEO 만난다",2022.02.03 17:22,박수형
98,https://zdnet.co.kr/view/?no=20220203161357,"""풀 알뜰폰 사업자 등장할 수 있는 제도 개선 필요""",2022.02.03 17:00,김태진


In [6]:
df.to_excel('./url.xlsx', index=False)

# url의 기사에 하나씩 접속하여 기사 본문 수집

In [7]:
title = []
sub_title = []
writer = []
date = []
category = []
content = []
tags = []
for link in tqdm(df["href"]):
    driver.get(link)
    driver.page_source
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try:
        title.append(
            soup.findAll(class_="news_head")[0].findAll("h1")[0].getText().strip()
        )
    except Exception:
        title.append("")
    try:
        sub_title.append(
            soup.findAll(class_="news_head")[0]
            .findAll(class_="summary")[0]
            .getText()
            .strip()
        )
    except Exception:
        sub_title.append("")
    try:
        writer.append(
            soup.findAll(class_="news_head")[0].findAll("a")[0].getText().strip()
        )
    except Exception:
        writer.append("")
    try:
        date.append(
            soup.findAll(class_="news_head")[0]
            .findAll("span")[0]
            .getText()
            .split("입력 :")[-1]
            .strip()
        )
    except Exception:
        date.append("")
    try:
        category.append(
            soup.findAll(class_="news_head")[0].findAll("a")[1].getText().strip()
        )
    except Exception:
        category.append("")
    try:
        content.append(
            "\n".join(
                [
                    x.getText()
                    for x in soup.findAll(id="content")[0].findAll("p")
                    if "byline" not in str(x) and len(x.getText()) != 0
                ]
            )
        )
    except Exception:
        content.append("")
    try:
        tags.append([x.getText() for x in soup.findAll(class_="tags")[0].findAll("a")])
    except Exception:
        tags.append("")

 34%|███▍      | 34/100 [05:55<11:30, 10.45s/it]


KeyboardInterrupt: ignored

In [ ]:
df1 = pd.DataFrame()
df1["title"] = title
df1["subtitle"] = sub_title
df1["writer"] = writer
df1["date"] = date
df1["category"] = category
df1["content"] = content
df1["tags"] = tags
df1

,title,subtitle,writer,date,category,content,tags
0,"KT스카이라이프, 산불피해 특별재난지역 방송통신 요금 감면","TV 무료 1개월, 인터넷 50% 할인",방송/통신,2022/03/08 10:00,,,"[KT스카이라이프, 산불]"
1,"국내 첫 유튜브 드라마, KT시즌에서 전편 무료","탑매니지먼트, 유튜브에서는 프리미엄 가입자만 시청 가능",방송/통신,2022/03/08 09:30,,,[시즌]
2,"HCN, 제2 개국 선언...""지역 1등 네트워크 플랫폼""",임직원에 새 비전 제시,방송/통신,2022/03/08 09:11,,,[HCN]
3,"KT, AI 코딩블록 교육 도구에 IoT 더했다",AI 코딩블록 사이트에서 자료 무료 제공,방송/통신,2022/03/08 09:00,,,"[KT, AI코딩블록 IoT키트]"
4,"""이통 3사 긴급구조 위치정보 품질 개선됐다""","방통위, LGU+ 보다 개선해야",방송/통신,2022/03/07 16:29 수정: 2022/03/07 17:03,,,"[방송통신위원회, 긴급구조, 위치정보]"
...,...,...,...,...,...,...,...
75,"KT AI 기가지니, 아마존 알렉사 품었다","""지니야~"" 부르면 한국어로 기가지니, ""알렉사~"" 부르면 영어로 아마존 알렉사",방송/통신,2022/02/10 08:52 수정: 2022/02/10 14:29,,,"[기가지니, 아마존 알렉사, KT, 인공지능, AI]"
76,"KT, 디지코 사업 매출 비중 절반까지 늘린다",2025년 디지털 플랫폼 + B2B 매출 비중 50% 목표,방송/통신,2022/02/09 17:13 수정: 2022/02/10 13:22,,,"[KT, 실적, 영업이익, 디지코, MNO, MVNO, 5G]"
77,"통신 3사, 올해 매출 총합 2.3兆 이상 성장 예고",[이슈진단+] 2021년 실적 발표에서 가이던스 제시,방송/통신,2022/02/09 16:12 수정: 2022/02/10 10:08,,,"[KT, SK텔레콤, LG유플러스, 실적, 매출, 매출 가이던스]"
78,"KT, 작년 4분기 영업익 3694억원...전년비 128.4%↑",자회사 자산 매각 일시적 증가분 반영,방송/통신,2022/02/09 14:37,,,"[KT, 실적, 디지코]"


In [ ]:
df1.to_excel('./content.xlsx', index=False)